In [1]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.spm as spm
from nipype.interfaces.base import CommandLine
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nibabel as nb
import os, re

200211-09:58:02,319 nipype.utils INFO:
	 Running nipype version 1.4.0-dev (latest: 1.4.1)


Gather necessary pre-requisites:

In [2]:
# get normalised GM/WM images to analyse in MultipleRegression model
# change norm_dir to vary between GM and WM analysis
norm_dir="/VBM/DARTEL_norm2mni_c1/"
images_mreg=[]

for i in os.listdir(norm_dir):
    match=re.match(r'(smwc.*\d{14,}.*.nii)', i)
    if match:
        images_mreg.append(os.path.join(norm_dir, match.group(0)))

print("Images where normalised only if included in study specific template!")
print(images_mreg, len(images_mreg))

# group them correctly
dep_norm=[]
con_norm=[]

for i in images_mreg:
    match_dep=re.match(r'.*DEP_.*.nii', i)
    match_con=re.match(r'.*CON_.*.nii', i)
    if match_dep:
        dep_norm.append(match_dep.group(0))
    elif match_con:
        con_norm.append(match_con.group(0))
    else:
        print("item in images_mreg does not match regex pattern! Investigate ... ")
    
print(dep_norm, len(dep_norm))
print(con_norm, len(con_norm))

Images where normalised only if included in study specific template!
['/VBM/DARTEL_norm2mni_c1/smwc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20160308162043.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20151216154455.nii', '/VBM/DARTEL_norm2mni_c1/smwc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141210170232.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20160310090712.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20151001173950.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20151009111760.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20150219142148.nii', '/VBM/DARTEL_norm2mni_c1/smwc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20150211171144.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20141208163153.nii', '/VBM/DARTEL_norm2mni_c1/smwc1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20160331140317.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20151001125857.nii', '/VBM/DARTEL_norm2mni_c1/smw

In [22]:
# calculate "global values" via nibabel objects
# form: total volume GM (WM, CSF?)

# find all the files starting with "c1" in all subdirs of the VBM directory via CommandLine interface from nipype
c1_files=CommandLine('find', args='/VBM/FED* -maxdepth 2 -type f -name c1*.nii', terminal_output='allatonce')
c1=c1_files.run()
# read the output (find stdout) line by line to list object
globals_files=c1.runtime.stdout.splitlines()

# create nibabel objects from all files
globals_nbobj=[nb.load(i) for i in globals_files]

# use nibabel object indexing etc. to calculate global means
# create list with fdata from all nbobjects
globals_nbfvals=[i.get_fdata() for i in globals_nbobj]



In [21]:
# change the printing option in numpy to 8 digits
np.set_printoptions(precision=8, suppress=True, threshold=np.inf)

# define show-slices function
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")

# use it to show centre slices of c1 images
#for i in globals_nbobj:
#    print(i.get_fdata())
#    plt.suptitle("values for c1 image")

In [42]:
# "//" for integer division
# think about how to weigh this and which voxels to include
print(np.sum(globals_nbfvals[0][:][:][:])//)

746565.8088570228


In [ ]:
# create multiple regression Node
mreg=Node(spm.MultipleRegressionDesign(), name="MultipleRegression")
mreg.inputs.in_files=
mreg.inputs.global_calc_values=
# use proportional global normalization
mreg.inputs.global_normalization=2
mreg.inputs.threshold_mask_absolute=
# think of mask image to threshold
#mreg.inputs.explicit_mask_file=
mreg.inputs.covariates=
results=mreg.run()
print(results.outputs)

Testing Zone: